# Create INCAR and INVICON MOLA JSON
version: 1

info:
- Create standard MOLA JSON

author: nuno costa

## MOLA Annotations Data Format

If you wish to combine multiple datasets, it is often useful to convert them into a unified data format. 

Objective: 

In [2]:

 #ANNOTATIONS FORMAT (BASED ON COCO)

 #Annotations format keys:

 { "info": None, 
   "licenses": [], TODO
   "categories": [], 
   "images": [],
   "annotations": [],
   "videos": [], TODO
   "tracks": [], TODO
   "segment_info": [], TODO
   "datasets": [{'name': 'INCAR', 'id': 1}, {'name': 'INVICON', 'id': 2}] 
 }

 #1 object definition:

 info{
     "year": int, 
     "version": str, 
     "description": str, 
     "contributor": str, 
     "url": str, 
     "date_created": datetime,
 }
 
 license{
     "id": int, 
     "name": str, 
     "url": str,
 }
 
 category{
     "id": int, 
     "name": str, 
     "supercategory": str,
 }
 

 image: {
     "id" : int,
     "video_id": int, #TODO
     "file_name" : str,
     "license" : int,
     # Redundant fields for COCO-compatibility
     "width": int,
     "height": int,
     "frame_index": int,
     "date_captured": datetime,
 
 annotation: {
     "category_id": int
     "image_id": int,
     "track_id": int,
     "bbox": [x,y,width,height],
     "area": float,
     # Redundant field for compatibility with COCO scripts
     "id": int,
     "iscrowd": 0 or 1,  (iscrowd=1) are used to label large groups of objects (e.g. a crowd of people)
     "segmentation": RLE(iscrowd=1) or [polygon](iscrowd=0), 
     
 }
 
 video: { #TODO
     "id": int,
     "name": str,
     "width" : int,
     "height" : int,
     "metadata": dict,  # Metadata about the video
 }
 
 segment{ #TODO
     "id": int, 
     "category_id": int, 
     "area": int, 
     "bbox": [x,y,width,height], 
     # Redundant field for compatibility with COCO scripts
     "iscrowd": 0 or 1,
 }
 
    
 track: { #TODO
     "id": int,
     "category_id": int,
     "video_id": int
 }

SyntaxError: invalid syntax (<ipython-input-2-580a7c10cdc0>, line 7)

## SETUP

In [1]:
from annotate_v5 import *
import platform 
import json
import os

In [19]:
#Define root dir dependent on OS
rdir='D:/external_datasets/MOLA/' #WARNING needs to be root datasets 
print('OS: {}'.format(platform.platform()))
if str(platform.platform()).upper().find('linux'.upper())>-1: rdir="/home/administrator/server_data_ext/Recordings/EASYRIDE/P19/" #'/mnt/d/external_datasets/'
print('root dir: {}'.format(rdir))
print('OS separator: {}'.format(os.path.sep))

OS: Linux-5.4.0-77-generic-x86_64-with-glibc2.10
root dir: /home/administrator/server_data_ext/Recordings/EASYRIDE/P19/
OS separator: /


In [3]:
res={
    'rgb': [2048, 1536], #w,h
    'thermal': [640,512],
    'pointcloud': [640,576]
}

## INIT motionLAB JSON
- uses annotate.init_json() function

In [9]:
molafile=rdir+'INCAR/'+'mola.json'
init_json(file=molafile)
molajson =  json.load(open(molafile))
molajson['datasets']=[{'name': 'INCAR', 'id': 1}]
with open(molafile, 'w') as f:
    json.dump(molajson, f)
for k in molajson:
    print(k, len(molajson[k]))

JSON INITIATED : D:/external_datasets/MOLA/INCAR/mola.json
info 5
licenses 0
categories 0
videos 0
images 0
tracks 0
segment_info 0
annotations 0
datasets 1


## IMPORT JSON LABELS

In [10]:
file="D:/external_datasets/MOLA/INCAR/20210506/Session 1/C8_P6_P5_1/gt/gt.json"
gt=json.load(open(file))
mergedjson = json.load(open(rdir+'/annotations/splitann_mola_fix_equal_reorder_cleanclass_cleanimg/mix_coco_and_tao_aggressive/test.json')) #EXAMPLES

In [31]:
def parse_path(path):
    parsed_path = path.replace('\\', '/')
    parsed_path = parsed_path.replace('\ ', '/')
    return parsed_path

In [32]:
def fix_pahts(gt):
    #fix gt datasource
    paths=gt['gTruth']['DataSource']
    if isinstance(paths, dict) and 'Source' in paths: paths=paths['Source']
    originalpath=paths[0]
    for p in paths:
        if p.find("gt") >-1 : 
            originalpath=p
            break
    originalpath=parse_path(originalpath)
    paths=[parse_path(p) for p in paths]
    paths = [os.path.join(*originalpath.split('/')[:-1], p.split('/')[-1]) if p.find("MATLAB") > -1 else p for p in paths]  #remove MATLAB BUG: 'C:\\Tools\\MATLAB\\R2020a\\examples\\symbolic\\data\\196.png'
    paths = [os.path.join(*p.split('/')[-7:]) for p in paths] #remove original 
    gt['gTruth']['DataSource']=paths
    return gt
gt=fix_pahts(gt)

#### CATEGORIES IMPORT

In [12]:
#EXAMPLE
display(mergedjson["categories"][0])

{'supercategory': 'sports', 'id': 1, 'name': 'aggressive', 'dataset': 1}

In [13]:
gt['gTruth']['LabelDefinitions']

[{'Name': 'VIOLENT',
  'Type': 'Scene',
  'LabelColor': [0, 0.7241, 0.6552],
  'Group': 'None',
  'Description': ''}]

In [21]:
def import_categories(molajson, gt, start_id=0):
    # IMPORT categories name and id
    cat_l=[]
    cat_l_id=[]
    cat_l_dset=[]
    cat=gt['gTruth']['LabelDefinitions']
    for i,c in enumerate(tqdm(cat)):
        cat_l.append(c['Name'])
        cat_l_id.append(start_id+i+1) # id start from 1
        cat_l_dset.append(1) # dataset index
        molajson['categories'].append({'name':cat_l[i],'id':cat_l_id[i],'dataset':cat_l_dset[i]})
    # ADDITIONAL CATEGORIES: MANUAL
    name='NONVIOLENT'
    cid=len(cat_l)+1
    dset=1
    molajson['categories'].append({'name':name,'id':cid,'dataset':dset})
    cat_l.append(name)
    cat_l_id.append(cid)
    cat_l_dset.append(dset)
    print("\n>> categories:\n", molajson['categories'][-2:])
    return molajson, cat_l, cat_l_id, cat_l_dset
molajson, cat_l, cat_l_id, cat_l_dset=import_categories(molajson, gt)

100%|██████████| 1/1 [00:00<00:00, 11428.62it/s]


>> categories:
 [{'name': 'VIOLENT', 'id': 1, 'dataset': 1}, {'name': 'NONVIOLENT', 'id': 2, 'dataset': 1}]


#### IMAGE IMPORT

In [15]:
#EXAMPLE
display(mergedjson["images"][0])

{'license': 3,
 'file_name': 'COCO/2017/images/train2017/000000391895.jpg',
 'coco_url': 'http://images.cocodataset.org/train2017/000000391895.jpg',
 'height': 360,
 'width': 640,
 'date_captured': '2013-11-14 11:18:45',
 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg',
 'id': 1,
 'caption': 'A man with a red helmet on a small moped on a dirt road. ',
 'dataset': 1}

In [16]:
gt['gTruth']['DataSource'][0]

'INCAR\\20210506\\Session 1\\C8_P6_P5_1\\gt\\rgb\\1.png'

In [22]:
def import_images(molajson, gt, start_id=0):
    # images filepath and id
    img_l=[]
    img_l_id=[]
    img=gt['gTruth']['DataSource']
    for i,im in enumerate(tqdm(img)):
        img_l.append(im)
        img_l_id.append(start_id+i+1) # id start from 1
        molajson['images'].append({'file_name':img_l[i],
                                   'id':img_l_id[i],
                                   'caption':img_l[i].split('/')[-4], # scenario
                                   'dataset':1})
    print("\n>> images:\n", molajson['images'][-2:])
    return molajson, img_l, img_l_id
molajson, img_l, img_l_id=import_images(molajson, gt)

100%|██████████| 580/580 [00:00<00:00, 140302.00it/s]


>> images:
 [{'file_name': 'INCAR/20210422/Session1/C10_P7_P8_1/gt/rgb/579.png', 'id': 579, 'caption': 'C10_P7_P8_1', 'dataset': 1}, {'file_name': 'INCAR/20210422/Session1/C10_P7_P8_1/gt/rgb/580.png', 'id': 580, 'caption': 'C10_P7_P8_1', 'dataset': 1}]


#### CREATE ANNOTATIONS

In [18]:
#EXAMPLE
display(mergedjson["annotations"][1000])

{'segmentation': [[619, 463, 761, 463, 761, 535, 619, 535]],
 'bbox': [619, 463, 142, 72],
 'area': 10224,
 'iscrowd': 0,
 'id': 1243036,
 'image_id': 146427,
 'category_id': 1,
 'track_id': 3385,
 '_scale_uuid': 'd83e8588-2744-4f9b-a14c-b6c8129cd696',
 'scale_category': 'moving object',
 'video_id': 626,
 'dataset': 2}

In [19]:
gt['gTruth']['LabelData'][0]

{'VIOLENT': False}

In [23]:
def create_annotations(molajson, gt, res, cat_l, cat_l_id, cat_l_dset, img_l_id, start_id=0):
    # annotations category_id, image_id, bbox, and dataset
    ann_id=[]
    ann_catid=[]
    ann_imgid=[]
    ann_bbox=[]
    ann_dset=[]
    labels=gt['gTruth']['LabelData']
    for i,l in enumerate(tqdm(labels)):
        annid=start_id+i+1
        catidx=cat_l.index("VIOLENT")
        if not l["VIOLENT"]: catidx=cat_l.index("NONVIOLENT")
        catid=cat_l_id[catidx]
        dataset=cat_l_dset[catidx]
        imgidx=i
        imgid=img_l_id[imgidx]
        bbox=[0, 0, res['rgb'][0], res['rgb'][1]] # [x,y,width,height], #default RGB
        area=res['rgb'][0]*res['rgb'][1] #default RGB
        ann_id.append(annid)
        ann_catid.append(catid)
        ann_imgid.append(imgid)
        ann_bbox.append(bbox)
        ann_dset.append(dataset)
        molajson['annotations'].append({'id':annid,
                                        'category_id':catid,
                                        'image_id':imgid,
                                        'bbox': bbox,
                                        'area': area,
                                        'iscrowd': 0,
                                        'dataset':dataset})
    print("\n>> annotations:\n", molajson['annotations'][-2:])
    return molajson, ann_id, ann_catid, ann_imgid, ann_bbox, ann_dset
molajson, ann_id, ann_catid, ann_imgid, ann_bbox, ann_dset=create_annotations(molajson, gt,res, cat_l, cat_l_id, cat_l_dset, img_l_id)

100%|██████████| 580/580 [00:00<00:00, 476905.77it/s]


>> annotations:
 [{'id': 579, 'category_id': 1, 'image_id': 579, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}, {'id': 580, 'category_id': 1, 'image_id': 580, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}]


# For loop script

In [54]:
datasets=['INCAR']
rdir="/home/administrator/server_data_ext/Recordings/EASYRIDE/P19/"
# FOR LOOP"
datasetsdir = os.listdir(rdir)
missing_gt_json=[]
missing_gt_mat=[]
label_folder="gt"
label_fname="gt.json"
label_mat_fname="gt.m"
for dataset in datasetsdir:
    if dataset in datasets:
        daysdir = os.path.join(rdir, dataset)
        if not os.path.isdir(daysdir): continue  # test if is a folder
        print(">>>\n EXTRACTING DATASET: "+dataset)
        #INIT JSON
        molafile=rdir+dataset+'/'+'mola.json'
        init_json(file=molafile)
        molajson =  json.load(open(molafile))
        molajson['datasets'] = [{'name': d, 'id': i+1} for i,d in enumerate(datasets)]
        with open(molafile, 'w') as f:
            json.dump(molajson, f)
        days = os.listdir(daysdir)
        imported_cats = False
        cat_start_id = 0
        img_start_id = 0
        ann_start_id = 0
        cat_l, cat_l_id, cat_l_dset = [], [], []
        img_l, img_l_id = [], []
        ann_id, ann_catid, ann_imgid, ann_bbox, ann_dset = [], [], [], [], []
        for day in days:
            sessiondir = os.path.join(daysdir, day)
            if not os.path.isdir(sessiondir): continue  # test if is a folder
            sessions = os.listdir(sessiondir)
            for session in sessions:
                scenariosdir = os.path.join(sessiondir, session)
                if not os.path.isdir(scenariosdir): continue  # test if is a folder
                scenarios = os.listdir(scenariosdir)
                for scenario in scenarios:
                    imgdir = os.path.join(scenariosdir, scenario)
                    if not os.path.isdir(imgdir): continue  # test if is a folder
                    labeldir = os.path.join(imgdir, label_folder)
                    # if not os.path.isdir(labeldir): continue #should exist
                    filename = os.path.join(labeldir, label_fname)
                    try:
                        gt = json.load(open(filename))
                    except:
                        print(">>>>>>>MISSING: ", filename)
                        missing_files.append(filename)
                        missing_gt_json.append(filename)
                        if not os.path.isfile(filename.replace(label_fname, label_mat_fname)): missing_gt_mat.append(filename.replace(label_fname, label_mat_fname))
                        continue
                    # fix gt paths
                    gt = fix_pahts(gt)
                    # update molajson
                    if not imported_cats:  # only imports one time
                        molajson, cat_l, cat_l_id, cat_l_dset = import_categories(molajson, gt, start_id=cat_start_id)
                        imported_cats = True
                    molajson, img_l, img_l_id = import_images(molajson, gt, start_id=img_start_id)
                    molajson, ann_id, ann_catid, ann_imgid, ann_bbox, ann_dset = create_annotations(molajson, gt, res,
                                                                                                    cat_l, cat_l_id,
                                                                                                    cat_l_dset, img_l_id,
                                                                                                    start_id=ann_start_id)
                    # update start ids to the last id
                    cat_start_id = cat_l_id[-1]
                    img_start_id = img_l_id[-1]
                    ann_start_id = ann_id[-1]

        # results
        for k in molajson:
            print(k, len(molajson[k]))

        # # Save
        print('\n >> SAVING...')
        jsonfile=molafile
        with open(jsonfile, 'w') as f:
            json.dump(molajson, f)
        with open(jsonfile.replace('.json', '_missing_gtmat.txt'),'w') as f:
            f.write(str(missing_gt_mat))
        with open(jsonfile.replace('.json', '_missing_gtjson.txt'),'w') as f:
            f.write(str(missing_gt_mat))
        print("JSON SAVED : {} \n".format(jsonfile))

        #retest results
        molajson =  json.load(open(molafile))
        for k in molajson:
            print(k, len(molajson[k]))

100%|██████████| 585/585 [00:00<00:00, 955850.35it/s]

>>>
 EXTRACTING DATASET: INCAR
JSON INITIATED : /home/administrator/server_data_ext/Recordings/EASYRIDE/P19/INCAR/mola.json

>> categories:
 [{'name': 'VIOLENT', 'id': 1, 'dataset': 1}, {'name': 'NONVIOLENT', 'id': 2, 'dataset': 1}]

>> images:
 [{'file_name': 'INCAR/20210422/Session1/C10_P7_P8_1/gt/rgb/579.png', 'id': 579, 'caption': 'C10_P7_P8_1', 'dataset': 1}, {'file_name': 'INCAR/20210422/Session1/C10_P7_P8_1/gt/rgb/580.png', 'id': 580, 'caption': 'C10_P7_P8_1', 'dataset': 1}]

>> annotations:
 [{'id': 579, 'category_id': 1, 'image_id': 579, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}, {'id': 580, 'category_id': 1, 'image_id': 580, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}]



100%|██████████| 585/585 [00:00<00:00, 735732.49it/s]



>> images:
 [{'file_name': 'INCAR/20210422/Session1/C10_P7_P8_2/gt/rgb/584.png', 'id': 1164, 'caption': 'C10_P7_P8_2', 'dataset': 1}, {'file_name': 'INCAR/20210422/Session1/C10_P7_P8_2/gt/rgb/585.png', 'id': 1165, 'caption': 'C10_P7_P8_2', 'dataset': 1}]

>> annotations:
 [{'id': 1164, 'category_id': 1, 'image_id': 1164, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}, {'id': 1165, 'category_id': 1, 'image_id': 1165, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}]

>> images:
 [{'file_name': 'INCAR/20210422/Session1/C10_P8_P7_1/gt/rgb/584.png', 'id': 1749, 'caption': 'C10_P8_P7_1', 'dataset': 1}, {'file_name': 'INCAR/20210422/Session1/C10_P8_P7_1/gt/rgb/585.png', 'id': 1750, 'caption': 'C10_P8_P7_1', 'dataset': 1}]

>> annotations:
 [{'id': 1749, 'category_id': 1, 'image_id': 1749, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}, {'id': 1750, 'category_id': 1, 'image_id': 1750, 'bbox': [0, 0, 2048, 1536], 'are

  0%|          | 0/590 [00:00<?, ?it/s]

...


100%|██████████| 590/590 [00:00<00:00, 353570.42it/s]



>> images:
 [{'file_name': 'INCAR/20210427/Session 2/C5_P16_P13_1/gt/rgb/589.png', 'id': 101315, 'caption': 'C5_P16_P13_1', 'dataset': 1}, {'file_name': 'INCAR/20210427/Session 2/C5_P16_P13_1/gt/rgb/590.png', 'id': 101316, 'caption': 'C5_P16_P13_1', 'dataset': 1}]

>> annotations:
 [{'id': 101315, 'category_id': 1, 'image_id': 101315, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}, {'id': 101316, 'category_id': 1, 'image_id': 101316, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}]

>> images:
 [{'file_name': 'INCAR/20210427/Session 2/C5_P16_P13_2/gt/rgb/590.png', 'id': 101906, 'caption': 'C5_P16_P13_2', 'dataset': 1}, {'file_name': 'INCAR/20210427/Session 2/C5_P16_P13_2/gt/rgb/591.png', 'id': 101907, 'caption': 'C5_P16_P13_2', 'dataset': 1}]

>> annotations:
 [{'id': 101906, 'category_id': 1, 'image_id': 101906, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}, {'id': 101907, 'category_id': 1, 'image_id': 1019

100%|██████████| 590/590 [00:00<00:00, 648660.38it/s]



>> images:
 [{'file_name': 'INCAR/20210430/Session 1/C11_P11_P10_2/gt/rgb/573.png', 'id': 115929, 'caption': 'C11_P11_P10_2', 'dataset': 1}, {'file_name': 'INCAR/20210430/Session 1/C11_P11_P10_2/gt/rgb/574.png', 'id': 115930, 'caption': 'C11_P11_P10_2', 'dataset': 1}]

>> annotations:
 [{'id': 115929, 'category_id': 1, 'image_id': 115929, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}, {'id': 115930, 'category_id': 1, 'image_id': 115930, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}]

>> images:
 [{'file_name': 'INCAR/20210430/Session 1/C12_P10_P11_1/gt/rgb/584.png', 'id': 116514, 'caption': 'C12_P10_P11_1', 'dataset': 1}, {'file_name': 'INCAR/20210430/Session 1/C12_P10_P11_1/gt/rgb/585.png', 'id': 116515, 'caption': 'C12_P10_P11_1', 'dataset': 1}]

>> annotations:
 [{'id': 116514, 'category_id': 1, 'image_id': 116514, 'bbox': [0, 0, 2048, 1536], 'area': 3145728, 'iscrowd': 0, 'dataset': 1}, {'id': 116515, 'category_id': 1, 'image_i

KeyError: 'gTruth'

In [28]:
molafile=rdir+'INCAR/'+'mola.json'
molajson =  json.load(open(molafile))
for k in molajson:
    print(k, len(molajson[k]))

info 5
licenses 0
categories 2
videos 0
images 1164
tracks 0
segment_info 0
annotations 1164
datasets 1
